### Importing Data

Country Codes: 

AT (Austria)
BE (Belgium)
CH (Switzerland)
CZ (Czech Republic)
DE-LU (Germany, Luxembourg)
DE-AT-LU (Germany, Austria, Luxembourg)
DK1 (Denmark 1)
DK2 (Denmark 2)
FR (France)
HU (Hungary)
IT-North (Italy North)
NL (Netherlands)
NO2 (Norway 2)
PL (Poland)
SE4 (Sweden 4)
SI (Slovenia)

Code for creating price dataframe:

In [14]:
def create_price_dataframe(country_code):
    import requests
    import json
    import pandas as pd
    url = "https://api.energy-charts.info/price?"

    location = "bzn=" + country_code
    start= "1546333200"
    stop= "1735635600"

    req = requests.get(url + location + "&start=" + start + "&end=" + stop)
    prices_data = req.json()
    prices_df = pd.DataFrame(prices_data["price"], prices_data["unix_seconds"]).reset_index()
    prices_df["datetime"]= pd.to_datetime(prices_df['index'],unit='s')
    prices_df.columns = ["unix", "prices", "datetime"]
    prices_df["month"] = prices_df["datetime"].dt.month
    prices_df["year"] = prices_df["datetime"].dt.year
    grouped_prices = prices_df.groupby(["year", "month"] )["prices"].mean()
    grouped_prices = pd.DataFrame(grouped_prices).reset_index()
    grouped_prices["date"] = grouped_prices["year"].astype(str) + "-" + grouped_prices["month"].astype(str)
    grouped_prices["date"] = pd.to_datetime(grouped_prices['date'])
    return grouped_prices


In [16]:
import pandas as pd
competitors = ["DE-LU", "DK1", "FR", "NL", "PL", "NO2"]
for region in competitors:
    df = create_price_dataframe(region)
    df.to_csv("spot_price_"+region, index=False)